In [ ]:
import pystac_client
from odc import stac as odc_stac
import numpy as np
import hvplot.xarray # noqa
import os
import matplotlib as mpl
from bokeh.models import CategoricalColorMapper, ColorBar

In [ ]:
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

In [ ]:
chunks = {"time": 1, "latitude": 1300, "longitude": 1300}

In [ ]:
crs = "EPSG:4326"
res = 1 / 12000  # 10 meter in degree

In [ ]:
time_range = "2022-10-11/2022-10-25"
minlon, maxlon = 12.3, 13.1
minlat, maxlat = 54.3, 54.6
bounding_box = [minlon, minlat, maxlon, maxlat]

In [ ]:
uri = "https://services.terrascope.be/stac/"
wcover_catalog = pystac_client.Client.open(uri)

In [ ]:
search = wcover_catalog.search(
    collections="urn:eop:VITO:ESA_WorldCover_10m_2021_AWS_V2",
    bbox=bounding_box,
)

items_wcover = search.item_collection()

In [ ]:
wcover_dc = (
    odc_stac.load(
        items_wcover,
        crs=crs,
        chunks=chunks,
        resolution=res,
        bbox=bounding_box,
    )
    .squeeze("time")
    .drop_vars("time")
    .rename_vars({"ESA_WORLDCOVER_10M_MAP": "wcover"})
)
wcover_dc

In [ ]:
cmap = mpl.colors.ListedColormap(
    [
        np.array([0, 0, 0]) / 255,
        np.array([0, 100, 0]) / 255,
        np.array([255, 187, 34]) / 255,
        np.array([255, 255, 76]) / 255,
        np.array([240, 150, 255]) / 255,
        np.array([250, 0, 0]) / 255,
        np.array([180, 180, 180]) / 255,
        np.array([240, 240, 240]) / 255,
        np.array([0, 100, 200]) / 255,
        np.array([0, 150, 160]) / 255,
        np.array([0, 207, 117]) / 255,
        np.array([250, 230, 160]) / 255,
    ]
)
bounds = [-5, 5, 15, 25, 35, 45, 55, 65, 75, 85, 92, 98, 105]
norm = mpl.colors.BoundaryNorm(np.array(bounds), cmap.N)
cblabels = [
    "no data",
    "tree cover",
    "shrubland",
    "grassland",
    "cropland",
    "built up",
    "bare/sparse vegetation",
    "snow and ice",
    "permanent water bodies",
    "herbaceous wetland",
    "mangroves",
    "moss and lichen",
]

In [ ]:
im = wcover_dc.wcover.plot.imshow(
    cmap=cmap,
    norm=norm,
    aspect=1.2,
    size=10,
    cbar_kwargs={
        "ticks": [0, 10, 20, 30, 40, 50, 60, 70, 80, 88.5, 95, 101.5],
        "spacing": "proportional",
    },
)
cb = im.colorbar
cb.set_ticklabels(cblabels)

In [ ]:
# https://discourse.holoviz.org/t/custom-discrete-colormaps/2183/15
def cbar_hook(hv_plot, _):
    plot = hv_plot.handles["plot"]
    factors = cblabels
    mapper = CategoricalColorMapper(
        palette=[mpl.colors.to_hex(i) for i in cmap.colors],
        factors=factors,
    )
    color_bar = ColorBar(color_mapper=mapper)
    plot.right[0] = color_bar


wcover_dc.hvplot.image(x="longitude", y="latitude", rasterize=True).opts(
    cmap=cmap,
    color_levels=bounds,
    clim=(bounds[0], bounds[-1]),
    frame_height=600,
    framewise=False,
    aspect="equal",
    hooks=[cbar_hook],
)